# SEMA-3D

The MIT License (MIT)

Copyright (c) 2016 AYLIEN

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

### Finetuning ESM-IF1 for epitope prediction tasks

In [4]:
import os
import copy
import math
import json
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
from esm.data import BatchConverter
from esm.inverse_folding.util import CoordBatchConverter
import torch
from torch.utils.data import Dataset
from torch import nn
from tqdm import tqdm
import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction
import scipy
import pandas as pd
import numpy as np
import pickle
from biotite.structure.residues import get_residues
import esm
import sklearn
from sklearn.metrics import r2_score, mean_squared_error, auc, plot_precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path




In [2]:
_, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()


def esmStructDataset(pdb_path):
    entity = pdb_path.name.split(".pdb")[0]
    pdb_id,chain,fab_chains = entity.split("_")
    if not os.path.exists(pdb_path):
        print("missing "+pdb_path)
        return
    print(pdb_path,chain.upper())
    structure = esm.inverse_folding.util.load_structure(str(pdb_path), chain.upper())
    resi_index = get_residues(structure)[0]
    resi_aa    = get_residues(structure)[1]
    resi_keys     = []
    cn =[]
    binary = []
    for resi_index_,resi_aa_ in zip(get_residues(structure)[0],get_residues(structure)[1]):
        key = (str(resi_aa_),resi_index_)
        cn.append(None)
        binary.append(None)
        resi_keys.append(key)    
    coords, seq = esm.inverse_folding.util.extract_coords_from_structure(structure)#structure)
    return {"pdb_id":pdb_id,"seq":seq,"chain":chain,"coords":coords,"cn":cn,"binary":binary,"residues":resi_keys}






/home/nikita/anaconda3/envs/inverse2/lib/python3.8/site-packages/esm/pretrained.py:172: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [3]:
if not os.path.exists("esm_structus.pkl"):
    path_1 = Path("../structures_antigen_fab/").glob("*.pdb")
    esm_structs = {}
    for pdb_path ina list(path_1):
        esm_structs[pdb_path.name.split(".pdb")[0]] = esmStructDataset(pdb_path)
    pickle.dump(esm_structs,  open("./esm_structus.pkl",'wb'))
esm_structs = pickle.load(open("./esm_structus.pkl",'rb'))
    

In [4]:
class epitopes_Dataset(Dataset):
    def __init__(self, epitope_data):
        self.epitope_data    = epitope_data
        #_, alphabet          = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
        self.batch_converter = CoordBatchConverter(alphabet)

    def __getitem__(self, idx):
        item = {}
        coords = self.epitope_data[idx]["coords"]
        seq    = self.epitope_data[idx]["seq"]     
        batch = [(coords, None, seq)]
        coords, confidence, strs, tokens, padding_mask = self.batch_converter(batch)
        item['coords']           = coords
        item['confidence']       = confidence
        item['tokens']           = tokens
        item['padding_mask']     = padding_mask
        item['labels_cn']        = torch.unsqueeze(torch.Tensor(self.epitope_data[idx]["cn"]),     0).to(torch.float32)
        item['labels_binary']    = torch.unsqueeze(torch.LongTensor(self.epitope_data[idx]["binary"]), 0)#.to(torch.float32)

        return item

    def __len__(self):
        return len(self.epitope_data)


def prepareEsmDataset(dataset, structs_data):
    dataset_esm = []
    for entity in dataset:
        pdb_id = entity["pdb_id"]
        
        assert pdb_id in structs_data
        struct_data = esm_structs[pdb_id]

        struct_data["cn"]            = [None for i in struct_data["residues"]]
        struct_data["binary"]        = [None for i in struct_data["residues"]]

        key_map = {(key[1],key[0]):i for i,key in enumerate(entity["keys"])} 

        for i,key in enumerate(struct_data["residues"]):
            key = (key[0],str(key[1]))
            if key not in key_map:
                struct_data["cn"][i]      = -100
                struct_data["binary"][i]  = -100
                continue
            struct_data["cn"][i]            = entity["cn"][key_map[key]]
            struct_data["binary"][i]        = entity["binary"][key_map[key]]

        if len(struct_data["seq"])>1500:
            print("Skip long  ",pdb_id,len(struct_data["seq"]))
            continue
        dataset_esm.append(struct_data)
    
    return epitopes_Dataset(dataset_esm)

In [8]:
class ESM1vForTokenClassification(nn.Module):
    def __init__(self, num_labels = 2):
        super().__init__()
        self.num_labels = num_labels    
        self.esm1v, self.esm1v_alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
        self.classifier = nn.Linear(512, self.num_labels)

    def forward(self, coords, padding_mask, confidence, tokens):
        prev_output_tokens = tokens[:, :-1]
        target = tokens[:, 1:]
        target_padding_mask = (target == alphabet.padding_idx)
        feat, x = self.esm1v.forward(coords, padding_mask, confidence, prev_output_tokens, features_only = True)
        f = feat[0,:,:]
        tt = torch.transpose(feat,1,2)
        logits = self.classifier(tt )
        return SequenceClassifierOutput(logits=logits)     

In [ ]:
def compute_metrics_quntative(p: EvalPrediction):
    preds = p.predictions[:,:,1]
    batch_size, seq_len = preds.shape
    out_labels, out_preds = [], []
    for i in range(batch_size):
        for j in range(seq_len):
            if p.label_ids[i, j] >= 0:
                out_labels.append(p.label_ids[i][j])
                out_preds.append(preds[i][j])
    out_labels_regr = [math.log(t+1) for t in out_labels ]
    return {
        "r2": r2_score(out_labels_regr,  out_preds),
        "mse": mean_squared_error(out_labels_regr,  out_preds)
    }

class MaskedMSELoss(torch.nn.Module):
    def __init__(self):
        super(MaskedMSELoss, self).__init__()
        
    def forward(self, inputs, target, mask):
        diff2 = (torch.flatten(inputs[:,:,1]) - torch.flatten(target)) ** 2.0 * torch.flatten(mask)
        result = torch.sum(diff2) / torch.sum(mask)
        if torch.sum(mask)==0:
            return torch.sum(diff2)
        else:
            return result

    
class MaskedClassRegressTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):        
        labels_cn    = inputs.pop("labels_cn")
        labels_bin   = inputs.pop("labels_binary")
        if "labels_binary_finite" in inputs:            
            labels_bin_finite     = inputs.pop("labels_binary_finite")
        outputs = model(**inputs)
        logits = outputs.logits
        masks    = ~torch.eq(labels_cn, -100).cuda()
        loss_fn  = MaskedMSELoss()
        loss_mse = loss_fn(logits, labels_cn, masks)
        loss     = loss_mse        
        return (loss, outputs) if return_outputs else loss
        
def collator_fn(x):
    if len(x)==1:
        return x[0]
    print('x:', x)
    return x

def model_init():
    return ESM1vForTokenClassification().cuda()

In [ ]:
def train_model(dataset_path = "dataset_4.5_10.0_12.0_12.0_CA_fab.pkl",
                dataset_folder = "./datasets_mono",
                mode = "binary", seed = 42):
    torch.cuda.empty_cache()
    isinf = False
    #dataset_path = f"{dataset_folder}/{dataset_path}"    
    d = pickle.load(open(dataset_path,'rb'))
    dataset_esm_test  = prepareEsmDataset(d["test"], esm_structs)
    dataset_esm_train = prepareEsmDataset(d["train"],esm_structs)
    cur_model = None
    trainer_ = MaskedClassRegressTrainer        
    training_args = TrainingArguments(
        output_dir='./results_fold_after_2020x' ,          # output directory
        num_train_epochs=2,#was 2              # total number of training epochs
        per_device_train_batch_size=1,   # batch size per device during training
        per_device_eval_batch_size=1,   # batch size for evaluation
        warmup_steps=0,                # number of warmup steps for learning rate scheduler
        learning_rate=1e-04,             # learning rate
        weight_decay=0.0,                # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=200,               # How often to print logs
        save_strategy = "no",
        do_train=True,                   # Perform training
        do_eval=False,                    # Perform evaluation
        evaluation_strategy="epoch",     # evalute after each epoch
        gradient_accumulation_steps=5,  # total number of steps before back propagation
        fp16=False,                       # Use mixed precision
        run_name="PDB_regr",      # experiment name
        seed=seed,                         # Seed for experiment reproducibility
        load_best_model_at_end=False,
        metric_for_best_model="eval_r2",
        greater_is_better=True,

    )
    cur_model = model_init()
    trainer = trainer_(
        model=cur_model,                 # the instantiated 🤗 Transformers model to be trained
        args=training_args,                   # training arguments, defined above
        train_dataset = dataset_esm_train,          # training dataset
        eval_dataset  = dataset_esm_test ,             # evaluation dataset
        data_collator = collator_fn
    )
    trainer.train()
    return cur_model,dataset_esm_test

def saveModel(ft_model, model_name):
    if not os.path.exists("./finetuned_models"):
        os.mkdir("./finetuned_models")
    best_model_wts = copy.deepcopy(ft_model.state_dict())
    pickle.dump(best_model_wts, open(f"./finetuned_models/{model_name}",'wb'))

def loadModel(path):
    model = model_init()
    wts   = pickle.load(open(path,'rb'))
    model.load_state_dict(wts)
    return model


In [ ]:
dataset_name = f"./datasets_mono/dataset_8.0_16.0_cn_aa.pkl"
inp_model, q_ = train_model(dataset_name,seed = 11111*i)
saveModel(inp_model,f"mse_8.0_16.0_ens_{i}")
    